In [67]:
import os
import cmdstanpy
from cmdstanpy import Model, StanFit, cmdstan_path
from cmdstanpy import Model, StanFit
import ujson as json

<h3>Example:</h3>

Running the model on Men's 2014 World Cup

In [68]:
with open('../data/worldcup_gelman/worldcup_2014.data.json', 'r') as fd:
    data_dict = json.load(fd)
data_dict.keys()

# stan_wc = Model(stan_file='../data/worldcup_gelman/worldcup_rep_ag.stan')
# stan_wc.compile()
# stan_wc

# worldcup_fit = stan_wc.sample(data=data_dict, chains=4)
# worldcup_fit.diagnose()
# worldcup_fit.summary()

dict_keys(['I', 'N', 'team_1', 'team_2', 'score_1', 'score_2', 'prior_score'])

In [69]:
with open('../data/worldcup_gelman/countries.data.json', 'r') as fd_country:
    country_dict = json.load(fd_country)
country_dict.keys()

dict_keys(['countries'])

<h3> Attempting the same for the Women' World Cup Data </h3>

(Import the data... and massaging it into the format stan needs it in.) 

Data input needs to be in a dictionary format;
Keys in the dictionary are the names of the variables in the data block of the stan model;

In [211]:
import pandas as pd
matches = pd.read_csv("/Users/nitya/NYCPyLadies_CmdStanPy/data/womens-world-cup-2019/wwc_matches.csv")
forecasts = pd.read_csv("/Users/nitya/NYCPyLadies_CmdStanPy/data/womens-world-cup-2019/wwc_forecasts.csv")
print(matches.columns, forecasts.columns)

Index(['date', 'league_id', 'league', 'team1', 'team2', 'spi1', 'spi2',
       'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2', 'score1',
       'score2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1', 'adj_score2'],
      dtype='object') Index(['forecast_timestamp', 'team', 'group', 'spi', 'global_o', 'global_d',
       'sim_wins', 'sim_ties', 'sim_losses', 'sim_goal_diff', 'goals_scored',
       'goals_against', 'group_1', 'group_2', 'group_3', 'group_4',
       'make_round_of_16', 'make_quarters', 'make_semis', 'make_final',
       'win_league', 'timestamp'],
      dtype='object')


In [212]:
matches.iloc[44:]['score1']

44   NaN
45   NaN
46   NaN
47   NaN
Name: score1, dtype: float64

In [195]:
#quarter-final info not yet filled in! Wikipedia to the rescue:[0,1,0,1]
matches.score1.iloc[[44, 45, 46, 47]] = [0,1,0,1]
matches.score2.iloc[[44, 45, 46, 47]] = [3,2,2,2]

In [196]:
matches.iloc[44:]

,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,probtie,proj_score1,proj_score2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
44,2019-06-27,352,FIFA Women's World Cup Final,Norway Women,England Women,84.10,92.65,0.3095,0.6905,0.0,0.81,1.47,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
45,2019-06-28,352,FIFA Women's World Cup Final,France Women,USA Women,96.79,98.50,0.4567,0.5433,0.0,1.25,1.41,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
46,2019-06-29,352,FIFA Women's World Cup Final,Italy Women,Netherlands Women,77.41,93.32,0.2146,0.7854,0.0,0.90,2.10,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
47,2019-06-29,352,FIFA Women's World Cup Final,Germany Women,Sweden Women,94.89,88.46,0.6271,0.3729,0.0,1.26,0.85,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [197]:
len(matches.team2.unique()), len(matches.team1.unique()), len(forecasts.team.unique())

(24, 24, 24)

In [198]:
#Getting the scores before the tournament
prior_tournament = forecasts[forecasts['forecast_timestamp']=='2019-05-16 18:22:11 UTC']
countries = prior_tournament[['team', 'spi']].sort_values(by=['spi'], ascending=False)
wwc2019_country = countries['team'].values
print(wwc2019_country)

['USA' 'France' 'Germany' 'Australia' 'Netherlands' 'England' 'Japan'
 'Canada' 'Brazil' 'Sweden' 'Spain' 'New Zealand' 'Norway' 'China'
 'South Korea' 'Italy' 'Nigeria' 'Cameroon' 'South Africa' 'Jamaica'
 'Scotland' 'Thailand' 'Chile' 'Argentina']


In [199]:
spi = dict(zip(wwc2019_country, list(range(1, len(wwc2019_country)+1))))

In [200]:
spi

{'USA': 1,
 'France': 2,
 'Germany': 3,
 'Australia': 4,
 'Netherlands': 5,
 'England': 6,
 'Japan': 7,
 'Canada': 8,
 'Brazil': 9,
 'Sweden': 10,
 'Spain': 11,
 'New Zealand': 12,
 'Norway': 13,
 'China': 14,
 'South Korea': 15,
 'Italy': 16,
 'Nigeria': 17,
 'Cameroon': 18,
 'South Africa': 19,
 'Jamaica': 20,
 'Scotland': 21,
 'Thailand': 22,
 'Chile': 23,
 'Argentina': 24}

In [201]:
import numpy as np
mean = np.mean(countries['spi'])
std = np.std(countries['spi'])
countries['prior_score'] = [(x - mean)/std for x in countries['spi']]

In [203]:
matches['team1'] = matches['team1'].apply(lambda x: x.split('Women')[0].rstrip())
matches['team2'] = matches['team2'].apply(lambda x: x.split('Women')[0].rstrip())

In [204]:
#matches = matches.replace('New', 'New Zealand')

In [205]:
matches['team1']

0           France 
1          Germany 
2            Spain 
3           Norway 
4        Australia 
5           Brazil 
6          England 
7        Argentina 
8           Canada 
9      New Zealand 
10           Chile 
11             USA 
12         Nigeria 
13         Germany 
14          France 
15       Australia 
16    South Africa 
17           Japan 
18         Jamaica 
19         England 
20     Netherlands 
21          Canada 
22          Sweden 
23             USA 
24           China 
25    South Africa 
26         Nigeria 
27     South Korea 
28           Italy 
29         Jamaica 
30        Scotland 
31           Japan 
32        Cameroon 
33     Netherlands 
34          Sweden 
35        Thailand 
36         Germany 
37          Norway 
38         England 
39          France 
40           Spain 
41          Sweden 
42           Italy 
43     Netherlands 
44          Norway 
45          France 
46           Italy 
47         Germany 
Name: team1, dtype: object

In [206]:
N = len(matches)
I = len(wwc2019_country)

In [207]:
mydict = dict({'I': I,
               'N': N,
                'team_1': matches['team1'].values,
               'team_2': matches['team2'].values,
               'score_1': matches['score1'].values,
               'score_2': matches['score2'].values,
              'prior_score': countries['prior_score'].values})

In [208]:
mydict['team_2']

array(['South Korea ', 'China ', 'South Africa ', 'Nigeria ', 'Italy ',
       'Jamaica ', 'Scotland ', 'Japan ', 'Cameroon ', 'Netherlands ',
       'Sweden ', 'Thailand ', 'South Korea ', 'Spain ', 'Norway ',
       'Brazil ', 'China ', 'Scotland ', 'Italy ', 'Argentina ',
       'Cameroon ', 'New Zealand ', 'Thailand ', 'Chile ', 'Spain ',
       'Germany ', 'France ', 'Norway ', 'Brazil ', 'Australia ',
       'Argentina ', 'England ', 'New Zealand ', 'Canada ', 'USA ',
       'Chile ', 'Nigeria ', 'Australia ', 'Cameroon ', 'Brazil ', 'USA ',
       'Canada ', 'China ', 'Japan ', 'England ', 'USA ', 'Netherlands ',
       'Sweden '], dtype=object)

In [209]:
mydict['team_1'] = [spi.get(x) for x in mydict['team_1']]
mydict['team_2'] = [spi.get(x) for x in mydict['team_2']]

In [210]:
mydict

{'I': 24,
 'N': 48,
 'team_1': [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 'team_2': [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 'score_1': array([ 4.,  1.,  3.,  3.,  1.,  3.,  2.,  0.,  1.,  0.,  0., 13.,  2.,
         1.,  2.,  3.,  0.,  2.,  0.,  1.,  3.,  2.,  5.,  3.,  0.,  0.,
         0.,  1.,  0.,  1.,  3., 

In [158]:
stan_wc = Model(stan_file='../data/worldcup_gelman/worldcup_rep_ag.stan')
stan_wc.compile()
stan_wc

INFO:cmdstanpy:compiling c++
INFO:cmdstanpy:compiled model file: /Users/nitya/NYCPyLadies_CmdStanPy/data/worldcup_gelman/worldcup_rep_ag


Model(name=worldcup_rep_ag,  stan_file="../data/worldcup_gelman/worldcup_rep_ag.stan", exe_file="/Users/nitya/NYCPyLadies_CmdStanPy/data/worldcup_gelman/worldcup_rep_ag")

In [159]:
worldcup_fit = stan_wc.sample(data=mydict, chains=4)

INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:finish chain 4


RuntimeError: Error during sampling, chain 0 returned error code 70, chain 1 returned error code 70, chain 2 returned error code 70, chain 3 returned error code 70

In [160]:
mydict

{'I': 24,
 'N': 48,
 'team_1': [2,
  3,
  11,
  13,
  4,
  9,
  6,
  24,
  8,
  None,
  23,
  1,
  17,
  3,
  2,
  4,
  None,
  7,
  20,
  6,
  5,
  8,
  10,
  1,
  14,
  None,
  17,
  None,
  16,
  20,
  21,
  7,
  18,
  5,
  10,
  22,
  3,
  13,
  6,
  2,
  11,
  10,
  16,
  5,
  13,
  2,
  16,
  3],
 'team_2': [None,
  14,
  None,
  17,
  16,
  20,
  21,
  7,
  18,
  5,
  10,
  22,
  None,
  11,
  13,
  9,
  14,
  21,
  16,
  24,
  18,
  None,
  22,
  23,
  11,
  3,
  2,
  13,
  9,
  4,
  24,
  6,
  None,
  8,
  1,
  23,
  17,
  4,
  18,
  9,
  1,
  8,
  14,
  7,
  6,
  1,
  5,
  10],
 'score_1': [4.0,
  1.0,
  3.0,
  3.0,
  1.0,
  3.0,
  2.0,
  0.0,
  1.0,
  0.0,
  0.0,
  13.0,
  2.0,
  1.0,
  2.0,
  3.0,
  0.0,
  2.0,
  0.0,
  1.0,
  3.0,
  2.0,
  5.0,
  3.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  3.0,
  0.0,
  2.0,
  2.0,
  0.0,
  0.0,
  3.0,
  1.0,
  3.0,
  2.0,
  1.0,
  1.0,
  2.0,
  2.0,
  0.0,
  1.0,
  0.0,
  1.0],
 'score_2': [0.0,
  0.0,
  1.0,
  0.0,
  2.0,
  0.0,
  1

In [ ]:
worldcup_fit.diagnose()

In [149]:
worldcup_fit.summary()

INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:finish chain 4


RuntimeError: Error during sampling, chain 0 returned error code 70, chain 1 returned error code 70, chain 2 returned error code 70, chain 3 returned error code 70